## 간단한 데이터 분석 워크플로우 정의해보기

### 1. 상태(State)와 환경(Import) 정의

In [1]:
import operator
import os
import warnings
from typing import Annotated, List, TypedDict

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from langchain_core.messages import AIMessage, HumanMessage, BaseMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import START, END, StateGraph

class AgentState(TypedDict):
    file_path: str
    messages: Annotated[List[BaseMessage], operator.add] # 메시지들의 리스트를 저장하며, 새로운 메시지가 추가될 때 기존 메시지들과 합쳐짐.

2. 데이터 분석 노드 정의(data_analyzer_node)

In [2]:
def data_analyzer_node(state: AgentState) -> dict:
    '''
    [노드 1: 데이터 탐색기]
    CSV를 읽고, 컬럼명을 정리한 후 데이터 프로필을 생성합니다.
    '''
    print("[단계 1/4] 데이터 탐색 시작...")

    file_path = state["file_path"] # 상태객체에서 파일경로 추출
    df = pd.read_csv(file_path) # pandasDML read_csv로 CSV파일을 DataFrame으로 로드

    analysis_summary = (
        f"데이터 파일명: {os.path.basename(file_path)}\n"
        f"데이터 형태: {df.shape}\n"
        f"데이터 타입: {df.dtypes}\n"
        f"기초 통계량: {df.describe()}\n"
        f"결측치 현황: {df.isnull().sum()}\n"
    ) # 데이터 분석 요약 생성

    return {"messages": [HumanMessage(content=f"데이터 파일의 구조와 주요 정보를 요약합니다.\n{analysis_summary}")]} # HumanMessage: 사용자가 보낸 메시지를 나타내는 객체, 분석결과를 메시지 형태로 포장해서 다음 노드로 전달.

### 3. 워크플로우 정의(그래프 생성 & 노드 등록)

In [3]:
# 워크플로우 그래프 생성. 어떤 작업들이 어떤 순서로 연결되어 있는지를 정의하는 틀을 만드는 것.
workflow = StateGraph(AgentState)

# 노드등록. analysis라는 이름으로 data_analyzer_node함수를 등록
workflow.add_node("analysis", data_analyzer_node)

workflow.add_edge(START, "analysis")

app = workflow.compile()
app

### 4. 실행코드

In [6]:
initial_state = {"file_path": "train_credit_v2.csv"}

for output in app.stream(initial_state):
    for message in output['analysis']['messages']:
        print(message.content)

[단계 1/4] 데이터 탐색 시작...
데이터 파일의 구조와 주요 정보를 요약합니다.
데이터 파일명: train_credit_v2.csv
데이터 형태: (10000, 18)
데이터 타입: UID                   object
주거 형태                 object
현재 직장 근속 연수           object
대출 목적                 object
대출 상환 기간              object
연간 소득                float64
체납 세금 압류 횟수            int64
개설된 신용계좌 수             int64
신용 거래 연수             float64
최대 신용한도              float64
신용 문제 발생 횟수            int64
마지막 연체 이후 경과 개월 수      int64
개인 파산 횟수               int64
현재 대출 잔액             float64
현재 미상환 신용액           float64
월 상환 부채액             float64
신용 점수                  int64
채무 불이행 여부              int64
dtype: object
기초 통계량:               연간 소득   체납 세금 압류 횟수    개설된 신용계좌 수      신용 거래 연수       최대 신용한도  \
count  1.000000e+04  10000.000000  10000.000000  10000.000000  1.000000e+04   
mean   2.238771e+06      0.201200     11.983500     20.105286  1.064346e+06   
std    1.768283e+06      0.444903      3.460264      6.793357  1.052856e+06   
min    2.676210e+05      0.000000  